In [10]:
import matplotlib.pyplot as plt
import keras_ocr
import cv2
import math
import numpy as np

In [6]:
!pip install keras-ocr

     -------------------------------------- 42.3/42.3 kB 171.4 kB/s eta 0:00:00
     ---------------------------------------- 9.5/9.5 MB 27.7 kB/s eta 0:00:00
  Using cached pyclipper-1.3.0.post3-cp39-cp39-win_amd64.whl (94 kB)
     ------------------------------------- 948.0/948.0 kB 50.4 kB/s eta 0:00:00
  Using cached editdistance-0.6.0-cp39-cp39-win_amd64.whl (24 kB)
  Using cached Shapely-1.8.4-cp39-cp39-win_amd64.whl (1.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19583 sha256=9b0664dcfc645f5058e0683c927e317cfa168ef15b8ef356890ab73002e78d87
  Stored in directory: c:\users\rutvik\appdata\local\pip\cache\wheels\2d\f0\a8\1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators
  Using cached keras_ocr-0.9.1-py3-none-any.whl (42 kB)
     -----------------

In [7]:
#General Approach.....
#Use keras OCR to detect text, define a mask around the text, and inpaint the
#masked regions to remove the text.
#To apply the mask we need to provide the coordinates of the starting and 
#the ending points of the line, and the thickness of the line

#The start point will be the mid-point between the top-left corner and 
#the bottom-left corner of the box. 
#the end point will be the mid-point between the top-right corner and the bottom-right corner.
#The following function does exactly that.
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

In [8]:
#Main function that detects text and inpaints. 
#Inputs are the image path and kreas_ocr pipeline
def inpaint_text(img_path, pipeline):
    # read the image 
    img = keras_ocr.tools.read(img_path) 
    
    # Recogize text (and corresponding regions)
    # Each list of predictions in prediction_groups is a list of
    # (word, box) tuples. 
    prediction_groups = pipeline.recognize([img])
    
    #Define the mask for inpainting
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in prediction_groups[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
        
        #For the line thickness, we will calculate the length of the line between 
        #the top-left corner and the bottom-left corner.
        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))
        
        #Define the line and inpaint
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,    
        thickness)
        inpainted_img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
                 
    return(inpainted_img)


In [ ]:
# keras-ocr will automatically download pretrained
# weights for the detector and recognizer.
pipeline = keras_ocr.pipeline.Pipeline()

img_text_removed = inpaint_text('trafic-sign.jpg', pipeline)

plt.imshow(img_text_removed)

cv2.imwrite('text_removed_image.jpg', cv2.cvtColor(img_text_removed, cv2.COLOR_BGR2RGB))

Looking for C:\Users\Rutvik\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\Rutvik\.keras-ocr\crnn_kurapan.h5
1/1 [==============================] - 86s 86s/step
